In [1]:
import matplotlib
from IPython.display import display
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [18, 20]

from pprint import pprint
from scipy.optimize import minimize, Bounds, LinearConstraint, fsolve
# from scipy.special import seterr
# seterr(singular="ignore")

import numpy as np
import copy
import matplotlib.gridspec as gridspec
import seaborn as sns; sns.set()
import pandas as pd
import time as time_lib
import os
from gurobipy import GRB, Model, quicksum

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Helper Classes and Functions

In [2]:
class SeabornFig2Grid():

    def __init__(self, seaborngrid, fig,  subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
            isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()

    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

    def _moveaxes(self, ax, gs):
        #https://stackoverflow.com/a/46906599/4124317
        ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax)
        self.fig.add_axes(ax)
        ax._subplotspec = gs
        ax.set_position(gs.get_position(self.fig))
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig)
        self.fig.canvas.mpl_connect("resize_event", self._resize)
        self.fig.canvas.draw()

    def _resize(self, evt=None):
        self.sg.fig.set_size_inches(self.fig.get_size_inches())
        
afmhot = matplotlib.cm.get_cmap('afmhot', 512)
newcolors = afmhot(np.linspace(0, 1, 512))
bg = np.array([200/256, 220/256, 250/256, 1])
newcolors[:1, :] = bg
afmhot_with0 = matplotlib.colors.ListedColormap(newcolors)

In [3]:
# chooses optimal random probabilities to optimize costs (note this only works for mu 1 vector, I believe)
def randOptimize():
    bounds = Bounds([1] + [0 for i in range(1, n)], [1 for i in range(n)])
    objective = lambda alpha: sum([c[j]*lamb[j]/(alpha[j]*mu[2*j]+(1-alpha[j+1])*mu[2*(j+1)-1]-lamb[j]) for j in range(n-1)]) + c[n-1]*lamb[n-1]/(alpha[n-1]*mu[2*(n-1)] - lamb[n-1])
    denom_constr = LinearConstraint([[0 for i in range(j)] + [mu[2*j], -mu[2*(j+1)-1]] + [0 for i in range(j+2, n)] for j in range(n-1)] + [[0 for j in range(n-1)] + [mu[2*(n-1)]]],
                                     [lamb[j] - mu[2*(j+1)-1] for j in range(n-1)] + [lamb[n-1]],
                                     [np.inf for j in range(n)])#, keep_feasible=True)
    res = minimize(objective, [1] + [1 for i in range(1, n)], method='trust-constr',
               options={'disp': False, 'xtol':1e-40,}, bounds=bounds, constraints=[denom_constr]) # 'xtol':1e-13, 
#     newfig = plt.figure()
#     plt.plot([i/1000 for i in range(int((1-epsilon/n)*1000)+1, 1001)], [objective([1, i/1000]) for i in range(int((1-epsilon/n)*1000)+1, 1001)])
#     display(newfig)
#     pprint(objective(res.x))
#     pprint(objective([1 for j in range(n)]))
    pprint(res.x)
    return(res.x)

# def randFunc(alpha):
#     term1 = [-c[0]*lamb[0]*(alpha[0]*mu[2*0]+(mu[2*0])/((1-alpha[0+1])*mu[2*(0+1)-1]-lamb[0])**2)] + [
#              -c[j]*lamb[j]*(alpha[j]*mu[2*j]+(mu[2*j])/((1-alpha[j+1])*mu[2*(j+1)-1]-lamb[j])**2) + -c[j-1]*lamb[j-1]*(alpha[j-1]*mu[2*(j-1)]+(-mu[2*j-1])/((1-alpha[j])*mu[2*j-1]-lamb[j-1])**2) for j in range(1, n-1)] + [
#              -c[n-1]*lamb[n-1]*((mu[2*(n-1)])/(alpha[n-1]*mu[2*(n-1)])**2)]
#     term2 = [alpha[n-1 + 0]*-mu[2*0]] + [
#              alpha[n-1 + j-1]*-mu[2*j-1] + alpha[n-1 + j]*-mu[2*j] for j in range (1, n)]# + [
# #              alpha[n-1 + n-1]*-mu[2*(n-1)]]
    
#     return [term1[i] + term2[i] - alpha[-1] for i in range(len(term1))] + [alpha[n-1 + j] * - (alpha[j]*mu[2*j] + (1-alpha[j+1]*mu[2*(j+1)-1])-lamb[j]) for j in range(n-1)] + [alpha[n-1 + n-1] * -(alpha[n-1]*mu[2*(n-1)]-lamb[n-1])] # + [alpha[0]-1]

# def randFunc(alpha):
#     term1 = [-c[0]*lamb[0]*mu[0]/(alpha[0]*mu[0]+(1-alpha[1])*mu[1]-lamb[0])**2] + [
#              -c[j]*lamb[j]*mu[2*j]/(alpha[j]*(mu[2*j])+(1-alpha[j+1])*mu[2*(j+1)-1]-lamb[j])**2 + -c[j-1]*lamb[j-1]*-mu[2*j-1]/(alpha[j-1]*mu[2*(j-1)]+(1-alpha[j])*mu[2*j-1]-lamb[j-1])**2 for j in range(1, n-1)] + [
#              -c[n-1]*lamb[n-1]*mu[2*(n-1)]/(alpha[n-1]*(mu[2*(n-1)])-lamb[n-1])**2 + -c[(n-1)-1]*lamb[(n-1)-1]*-mu[2*(n-1)-1]/(alpha[(n-1)-1]*mu[2*((n-1)-1)]+(1-alpha[n-1])*mu[2*(n-1)-1]-lamb[(n-1)-1])**2]
#     term2 = [alpha[n-1 + 0]*-mu[0]] + [
#              alpha[n-1 + j-1]*mu[2*j-1] + alpha[n-1 + j]*-mu[2*j] for j in range(1, n)]
#     term3 = [alpha[2*(n-1) + j] for j in range(n)]
#     term4 = [-alpha[3*(n-1) + j] for j in range(n)]
#     term5 = [alpha[-1]] + [0 for i in range(1, n)]
#     stationarity = [term1[i] + term2[i] + term3[i] + term4[i] + term5[i] for i in range(len(term1))]
    
#     comp_slack = [alpha[n-1 + j] * - (alpha[j]*mu[2*j] + (1-alpha[j+1])*mu[2*(j+1)-1]-lamb[j]) for j in range(n-1)] + [
#                   alpha[n-1 + n-1] * -(alpha[n-1]*mu[2*(n-1)]-lamb[n-1])] + [
#                   alpha[2*(n-1) + j] * alpha[j] for j in range(n)] + [
#                   alpha[3*(n-1) + j] * - alpha[j] for j in range(n)]
    
    
#     return stationarity + comp_slack + [alpha[0]-1]


# def randOptimize():
#     init = 4 * [1 for j in range(n)] + [1]
# #     func_sum = lambda alpha: [sum([-c[j]*lamb[j] * (alpha[j]*mu[2*j] + (1-alpha[j+1])*mu[2*(j+1)-1]-lamb[j])**(-2) for j in range(n-1)]) + c[n-1]*mu[n-1] * (alpha[n-1]*mu[2*(n-1)]-lamb[n-1])**(-2)*mu[2*(n-1)] + sum([alpha[n-1 + j] * (-mu[2*j] - (-mu[2*(j+1)-1])) for j in range(n-1)]) + alpha[n-1 + n-1] * (- mu[2*(n-1)]) -alpha[-1],
                              
                              
    
#     root = fsolve(randFunc, init, xtol=1e-25, factor = 1e-4) #, method='trust-constr', options={'disp': False, 'xtol':1e-40,}, bounds=bounds, constraints=[denom_constr]) # 'xtol':1e-13, 
#     pprint(list(root[:n]))
#     pprint(randFunc(root))
#     return([1] + list(root[1:n]))


def MaxWeight():
#     objective = lambda decision: sum([gamma[j] * Lq[j] ** beta * ((decision[j] <= 0.5)*mu[2*j] + (decision[j+1] > 0.5)*mu[2*(j+1)-1]) for j in range(n-1)]) + gamma[n-1] * Lq[n-1] ** beta * ((decision[n-1] > 0.5)*mu[2*(n-1)])
#     bounds = Bounds([1] + [0 for i in range(1, n)], [1 for i in range(n)])
#     res = minimize(objective, [1] + [0 for j in range(1, n)], method='trust-constr',
#                   options={'xtol':1e-13, 'disp': False}, bounds=bounds)
#     return [int(i>0.5) for i in res.x]
    # create the model
    d = Model('MaxWeight Decisions')
    d.setParam("LogToConsole", 0)

    # %% add decision variables
    x = d.addVars(range(n), vtype = GRB.BINARY if time_mode == "continuous" else GRB.CONTINUOUS, name = 'x')

    # %% add constraints
    d.addConstr((x[0] == 1), name = "first server serves first queue")

    # define objective
    if time_mode == "continuous":
        d.setObjective(quicksum([gamma[j] * Lqs[alg_index][j] ** beta * ((x[j])*mu[2*j] + (1-x[j+1])*mu[2*(j+1)-1]) for j in range(n-1)]) + gamma[n-1] * Lqs[alg_index][n-1] ** beta * ((x[n-1])*mu[2*(n-1)]), GRB.MAXIMIZE)
    else:
        d.setObjective(quicksum([gamma[j] * Lqs[alg_index][j] ** beta * ((x[j])*mu[2*j] + (1-x[j+1])*mu[2*(j+1)-1]) for j in range(n-1)]) + gamma[n-1] * Lqs[alg_index][n-1] ** beta * ((x[n-1])*mu[2*(n-1)]), GRB.MAXIMIZE)
    # %% perform optimization
    d.optimize()

    result = []
    for j in range(n):
        result.append(int(x[j].x))
        
    return result



def Reward(state):
    return sum([-c[j] * state[j] for j in range(n)])

def TransitionProb(next_state, current_state, actions):
    if sum([abs(current_state[j] - next_state[j]) for j in range(n)]) != 1:
        return 0
    
    denominator = sum(lamb) + (current_state[0] > 0) * mu[0] + sum([(current_state[j] > 0 and actions[j+1] == 0) * mu[2*(j+1)-1] +
                      (current_state[j+1] > 0 and actions[j+1] == 1) * mu[2*(j+1)] for j in range(n-1)])        
    for j in range(n):
        if next_state[j] < 0 or current_state[j] < 0:
            return 0
        if next_state[j] > current_state[j]:
            return lamb[j] / denominator
        elif next_state[j] < current_state[j] and j < n-1:
            return ((actions[j] == 0) * mu[2*j] +
                    (actions[j+1] == 1) * mu[2*(j+1)-1]) / denominator
        elif next_state[j] < current_state[j] and j == n-1:
            return ((actions[j] == 1) * mu[2*j]) / denominator


def VStar(state, gamma_exp):
    gamma_exp += 1
    if gamma_exp < gamma_max:
        next_states = []
        for j in range(n):
            changes = [1]
            if state[j] > 0:
                changes.append(-1)
            for change in changes:
                next_state = copy.deepcopy(state)
                next_state[j] += 1
                if tuple(next_state) not in state_values:
                    state_values[tuple(next_state)] = VStar(next_state, gamma_exp)
                next_states.append(tuple(next_state))
        options = []
        for j in range(2 ** (n-1)):
            action = [1] + [0 for i in range((n-1) - len(bin(j)[2:]))] + [int(i) for i in bin(j)[2:]]
            options.append(sum([TransitionProb(copy.deepcopy(next_state), copy.deepcopy(state), action) * state_values[next_state] for next_state in next_states]))
#         print("did one")
        return Reward(state) + gamma * max(options)
    else:
        return 0

# Input Parameters

In [4]:
ns = [2] # number of servers/buffers
mu = [1 for i in range(2*max(ns)-1)] # service rates per server
duration = 5*10**8 # number of events (cont) or time units (disc)
epsilons = [0.005] # extent to which the system is in heavy traffic [0.1, 0.05, 0.01, 0.005, 0.001]
c = list(range(1, 20)) #if max(ns)==2 else np.random.randint(5, 15, size=max(ns)) # random costs per buffer, comment out if keeping costs between algs
time_mode = "continuous"
out_folder = "Output\\N-Systems" # base folder
#"C:\\Users\\Adam\\Dropbox\\REU 2021 Shared Folder\\Adam\\Code\\Output\\N-Systems"
additional_info = "" # info to write to the run log # distribution of q0 and q1-q0
lamb_defined = True                                 if max(ns)==2 else True # set all lambda equal to 1-(epsilon/n)?
ordered_costs = True # sort costs? will try ascending and descending
warmup_period = 0.4 # proportion of duration to cut off
graph = False # create graphs? very taxing on memory
graph_dist = False
to_folder = True
show_output = True # print output in Jupyter Notebook?
gamma = 0.8
gamma_max = 100


directions = ["crp_tri", "crp_rect", "noncrp_tri", "noncrp_rect"] if not lamb_defined else [1]
algs = ["MDP"]#["random", "random_idle"]#["maxweight", "weighted_maxweight", "cmu", "cmu_thresh", "random", "random_idle"] # ["maxweight", "weighted_maxweight", "cmu", "cmu_single_thresh", "cmu_multi_thresh", "random", "prioritize_right", "threshold", "random_idle"]
num_algs = len(algs)
color_dict = {i:color for (i, color) in enumerate([f"C{i}" for i in range(10)])}
# maxweight params
# gamma = [1 for j in range(n)] # "cost"
beta = 1

input_params = {"mu":mu, "duration":duration, "warmup proportion":warmup_period, "cost vect":c, "beta":beta, "time":time_mode,
                "lamb = 1-epsilon/n":lamb_defined, "random cost order":(not ordered_costs), "additional info":additional_info} # "gamma":gamma, 

# Simulation

In [5]:
out_time = int(time_lib.time())
# out_i = 0
# if to_folder:
#     os.makedirs(f"{out_folder}\\{out_time}")
# with open(f"{out_folder}\\{out_time}\\"*to_folder + "info.txt", "w") as file:
#     for (label, data) in input_params.items():
#         print(f"{label}:", data, file=file)
    
    
for case in range(1 + ordered_costs):
    if ordered_costs:
        c.sort()
        if case == 0:
            if show_output:
                print("Good Case: Queue 0 is least expensive.")
        else:
            continue
            if show_output:
                print("Bad Case: Queue 0 is most expensive.")
            c = c[::-1]
#     if type(gamma) == str and gamma == "cost":
#         gamma = c
        
    for n in ns:
        
        for direction in directions:
            lambdas = []

            if not lamb_defined:

                if n == 2:
                    fig = plt.figure(figsize=[10, 10])
                    # plot capacity region w/ lambdas approaching facets
                    plt.fill_between([0, mu[0]], 0, mu[2], color='C2', alpha=0.1)
                    plt.fill_between([mu[0], mu[0] + mu[1]], 0, [mu[2], 0], color='C2', alpha=0.1)
                    plt.axhline(mu[2], xmin=0, xmax=mu[0])
                    plt.axvline(mu[0], ymin=0, ymax=mu[2], linestyle="dashed")
                    plt.plot([0, mu[0]+mu[1]], [mu[2] + mu[0]*mu[2]/mu[1], 0])
                    plt.xlim([0, (mu[0]+mu[1])])
                    plt.ylim([0, (mu[2] + mu[0]*mu[2]/mu[1])])

                    # what direction do we approach heavy traffic limit
                    if direction == "noncrp_rect":
                        slope = mu[2] / mu[0]
                        for epsilon in epsilons:
                                lambdas.append((mu[0] * (1-epsilon/2), slope * mu[0] * (1-epsilon/2)))
                                plt.plot(lambdas[-1][0], lambdas[-1][1], marker="+")
                    if direction == "crp_tri":
                        slope = (mu[0]+mu[1]) / (mu[2] + mu[0]*mu[2]/mu[1])
                        intercept = mu[2]/2 - (mu[0]+mu[1]/2) * slope
                        for epsilon in epsilons:
                                lambdas.append((mu[0]+ mu[1] / 2 * (1-epsilon/2), intercept + slope * (mu[0]+mu[1]/2) * (1-epsilon/2)))
                                plt.plot(lambdas[-1][0], lambdas[-1][1], marker="+")
                    if direction == "crp_rect":
                        for epsilon in epsilons:
                                lambdas.append((3*mu[0]/4, mu[2] * (1-epsilon/10)))
                                plt.plot(lambdas[-1][0], lambdas[-1][1], marker="+")
                    if direction == "noncrp_tri":
                        for epsilon in epsilons:
                                lambdas.append((mu[0] + epsilon*2 * mu[1]/2, mu[2] * (1-epsilon*2)))
                                plt.plot(lambdas[-1][0], lambdas[-1][1], marker="+")

                    plt.xlabel("Buffer 0 Arrival Rate")
                    plt.ylabel("Buffer 1 Arrival Rate")
                    plt.title("Capacity Region")
                    plt.savefig(f"{out_folder}\\{out_time}\\"*to_folder + f"region_{direction}.png")
                    if show_output:
                        plt.show()
                        print("\n\n")

                else:
                    # randomly select lambdas that will be stable
                    for i in range(len(algs)):
                        lamb = [np.random.uniform(mu[0], 1.5*mu[0])]
                        for i in range(1, n):
                            lamb.append(np.random.uniform(0.75 * (mu[2*i]-(lamb[i-1]-mu[2*(i-1)])), mu[2*i]-(lamb[i-1]-mu[2*(i-1)])))
                        lambdas.append(lamb)

            else:
                for epsilon in epsilons:
                    lambdas.append([1-epsilon for j in range(n)])

            for lamb in lambdas:
    #             if n == 2:
    #                 epsilon = abs((1-lamb[1]/mu[2]) - (lamb[0]-mu[0])/mu[1])
    #             else:
    #             epsilon = 1 - sum(lamb) / sum(mu[::2])
                if not lamb_defined:
                    epsilon = sum(mu[::2]) - sum(lamb)
                else:
                    epsilon = epsilons[lambdas.index(lamb)]
    #             print(epsilon)


                # initializing variables
#                 if time_mode == "continuous":
#                     rng1 = np.random.default_rng(100)
#                     rng2 = np.random.default_rng(200)
#                     rng3 = np.random.default_rng(300)
#                 elif time_mode == "discrete":
#                     rng1 = [np.random.default_rng(100 + j) for j in range(n)]
#                     rng2 = [np.random.default_rng(200 + act) for act in range(2*n-1)]
#                     rng3 = np.random.default_rng(300)
#                     unused_tallys = [[0 for j in range(n)] for alg_index in range(num_algs)]

                # for threshold algs
#                 Lr = [[110, 330][lambdas.index(lamb)] for j in range(n-1)]#[c[j+1]/c[j] * 120 * np.log(1/epsilon) for j in range(n-1)] # used to be 75
                # for best random alg
#                 alpha = randOptimize()
#                 pprint(alpha)
#                 pprint(lamb)

                # initializing statistics
                if graph:
                    event_time_stats = [[] for alg_index in range(num_algs)]
                    Lq_stats = [[] for alg_index in range(num_algs)]
                if graph_dist:
                    Lq_freqs = [{} for alg_index in range(num_algs)]
                decisions = [[1 for i in range(n)] for alg_index in range(num_algs)]
                Lqs = [[0 for i in range(n)] for alg_index in range(num_algs)]
                times = [0 for alg_index in range(num_algs)]
                Lq_tallys = [[0 for j in range(n)] for alg_index in range(num_algs)]
                
#                 sum_Lq_tallys = [0 for alg_index in range(num_algs)]
#                 sum_weighted_Lq_tallys = [0 for alg_index in range(num_algs)]

                state_values = {}
                print(-VStar([200 for j in range(n)], 0))

#                 # initialize subplots and table
#                 avg_df = pd.DataFrame(columns=["Graph Index", "Direction", "epsilon", "alpha", "Case", "Algorithm"] + [f"lambda_{j}" for j in range(max(ns))] + [f"Avg Lq_{j}" for j in range(max(ns))] + ["Avg sum(Lq)", "Avg Cost-Weighted sum(Lq)"]) # index=range(len(algs)), 
#                 if graph:
#                     fig_Lq, axs_Lq = plt.subplots(len(algs) // 2, 2, sharex=False, sharey=False)


                
#                 # initialize joint dist plots
#                 if graph_dist:
#                     fig_dist = plt.figure()
#                     subfigs_dist = fig_dist.subfigures(len(algs) // 2, 2, wspace=0.07)
#                     fig_dist_alt = plt.figure()
#                     subfigs_dist_alt = fig_dist_alt.subfigures(len(algs) // 2, 2, wspace=0.07)
                            
#                 # plot queue lengths
#                 for alg_index in range(num_algs):
#                     alg = algs[alg_index]
#                     if graph:
#                         for j in range(n):
#                             axs_Lq[alg_index // 2, alg_index % 2].plot(event_time_stats[alg_index], [Lq[j] for Lq in Lq_stats[alg_index]], label = f"Lq_{j}", color=color_dict[j])
#                         if "thresh" in alg:
#                             for j in range(n-1):
#                                 axs_Lq[alg_index // 2, alg_index % 2].axhline(Lr[j], xmin=0, xmax=times[alg_index], label = f"Lr_{j}", linestyle="dotted", color=color_dict[j], alpha=0.6)
#                         axs_Lq[alg_index // 2, alg_index % 2].legend()
#                         axs_Lq[alg_index // 2, alg_index % 2].set(ylabel="Queue Length", xlabel="Time")
#                         axs_Lq[alg_index // 2, alg_index % 2].set_title(alg)

#                     if graph_dist:
# #                         pprint([Lq_0 for (Lq_0, Lq_1) in Lq_freqs[alg_index].keys()])
# #                         pprint([Lq_1 for (Lq_0, Lq_1) in Lq_freqs[alg_index].keys()])
# #                         pprint(Lq_freqs[alg_index].values())
#                         df = pd.DataFrame({"Lq_0":[Lq_0 for (Lq_0, Lq_1) in Lq_freqs[alg_index].keys()], "Lq_1":[Lq_1 for (Lq_0, Lq_1) in Lq_freqs[alg_index].keys()], "weights":list(Lq_freqs[alg_index].values())})
#                         x_range = (min(df["Lq_0"]), max(df["Lq_0"]))
#                         y_range = (min(df["Lq_1"]), max(df["Lq_1"]))
#                         x_bins = min(50, len(df["Lq_0"]))
#                         y_bins = min(50, len(df["Lq_1"]))
#                         axs = subfigs_dist[alg_index // 2, alg_index % 2].subplots(2, 2, sharex=False, sharey=False, gridspec_kw={'height_ratios': [1, 3], 'width_ratios': [3, 1], 'wspace':0, 'hspace':0})
#                         axs[1, 0].hist2d(df["Lq_0"], df["Lq_1"], weights=df["weights"], bins=(x_bins, y_bins), range=(x_range, y_range), density=False, cmin=0, cmap=afmhot_with0)
#                         axs[1, 0].set(ylabel="Lq_1", xlabel="Lq_0")
#                         axs[0, 0].hist(df["Lq_0"], weights=df["weights"], bins=x_bins, range=x_range, density=True)
#                         axs[0, 0].set_xticklabels([])
#                         axs[0, 0].set_yticklabels([])
#                         axs[1, 1].hist(df["Lq_1"], weights=df["weights"], bins=y_bins, range=y_range, density=True, orientation="horizontal")
#                         axs[1, 1].set_xticklabels([])
#                         axs[1, 1].set_yticklabels([])
#                         subfigs_dist[alg_index // 2, alg_index % 2].suptitle(alg)
#                         subfigs_dist[alg_index // 2, alg_index % 2].delaxes(axs[0, 1])
        
#                         x_range = (min(df["Lq_0"]), max(df["Lq_0"]))
#                         y_range = (min([df["Lq_1"][i] - df["Lq_0"][i] for i in range(len(df))]), max([df["Lq_1"][i] - df["Lq_0"][i] for i in range(len(df))]))
#                         x_bins = min(50, len(df["Lq_0"]))
#                         y_bins = min(50, len(df["Lq_1"]))
#                         axs_alt = subfigs_dist_alt[alg_index // 2, alg_index % 2].subplots(2, 2, sharex=False, sharey=False, gridspec_kw={'height_ratios': [1, 3], 'width_ratios': [3, 1], 'wspace':0, 'hspace':0})
#                         axs_alt[1, 0].hist2d(df["Lq_0"], [df["Lq_1"][i] - df["Lq_0"][i] for i in range(len(df))], weights=df["weights"], bins=(x_bins, y_bins), range=(x_range, y_range), density=False, cmin=0, cmap=afmhot_with0)
#                         axs_alt[1, 0].set(ylabel="Lq_1-Lq_0", xlabel="Lq_0")
#                         axs_alt[0, 0].hist(df["Lq_0"], weights=df["weights"], bins=x_bins, range=x_range, density=True)
#                         axs_alt[0, 0].set_xticklabels([])
#                         axs_alt[0, 0].set_yticklabels([])
#                         axs_alt[1, 1].hist([df["Lq_1"][i] - df["Lq_0"][i] for i in range(len(df))], weights=df["weights"], bins=y_bins, range=y_range, density=True, orientation="horizontal")
#                         axs_alt[1, 1].set_xticklabels([])
#                         axs_alt[1, 1].set_yticklabels([])
#                         subfigs_dist_alt[alg_index // 2, alg_index % 2].suptitle(alg)
#                         subfigs_dist_alt[alg_index // 2, alg_index % 2].delaxes(axs_alt[0, 1])

#                     # populate avgs table with run results
#                     if graph or graph_dist:
#                         avg_df.loc[alg_index, "Graph Index"] = out_i
#                     if direction != 1:
#                         avg_df.loc[alg_index, "Direction"] = direction
#                     avg_df.loc[alg_index, "epsilon"] = epsilon
#                     if "random" in alg:
#                         avg_df.at[alg_index, "alpha"] = alpha
#                     if ordered_costs:
#                         avg_df.loc[alg_index, "Case"] = "good" if (case == 0) else "bad"
#                     avg_df.loc[alg_index, "Algorithm"] = alg
#                     for j in range(n):
#                         avg_df.loc[alg_index, f"lambda_{j}"] = lamb[j]
#                         avg_df.loc[alg_index, f"Avg Lq_{j}"] = Lq_tallys[alg_index][j] #np.mean([Lq[j] for Lq in Lq_stat])
#                     avg_df.loc[alg_index, "Avg sum(Lq)"] = sum(Lq_tallys[alg_index])#sum_Lq_tallys[alg_index]/(times[alg_index]) #np.mean([sum(Lq) for Lq in Lq_stat])
#                     avg_df.loc[alg_index, "Avg Cost-Weighted sum(Lq)"] = sum([c[j] * Lq_tallys[alg_index][j] for j in range(n)]) #sum_weighted_Lq_tallys[alg_index]/(times[alg_index]) # np.mean([sum([c[i] * Lq[i] for i in range(n)]) for Lq in Lq_stat])

#                 # display or save multigraphs and tables w/ all algorithm outputs
# #                 os.makedirs(f"{out_folder}\\{out_time}\\{out_i}")
# #                 result_dict = {"epsilon":round(epsilon, 4), "direction":direction, "good case":(case == 0)}
# #                 with open(f"{out_folder}\\{out_time}\\{out_i}\\info.txt", "w") as file:
# #                     for (label, data) in result_dict.items():
# #                         print(f"{label}:", data, file=file)
# #                     print(avg_df, file=file)
# #                     print(list(avg_df["Avg Cost-Weighted sum(Lq)"]), file=file)
                
# #                 csv_filename = f"{out_folder}\\{out_time}\\{str(round(epsilon, 5)).replace('.', '_')}.csv"
# #                 avg_df.to_csv(csv_filename, mode="a", header= (not os.path.exists(csv_filename)), index=False)
#                 csv_filename = f"{out_folder}\\{out_time}\\"*to_folder + "results.csv"
#                 avg_df.to_csv(csv_filename, mode="a", header= (not os.path.exists(csv_filename)), index=False)


#                 if graph:
#                     fig_Lq.suptitle(f"Epsilon {round(epsilon, 3)} Queue Length", fontsize="xx-large", y=0.94)
#                     fig_Lq.savefig(f"{out_folder}\\{out_time}\\"*to_folder + f"{out_i}_Lq.png", bbox_inches="tight")
#                 if graph_dist:
#                     fig_dist.suptitle(f"Epsilon {round(epsilon, 3)} Joint Distribution", fontsize="xx-large", y=0.94)
#                     fig_dist.savefig(f"{out_folder}\\{out_time}\\"*to_folder + f"{out_i}_dist.png")
                    
#                     fig_dist_alt.suptitle(f"Epsilon {round(epsilon, 3)} Joint Distribution (Alternate)", fontsize="xx-large", y=0.94)
#                     fig_dist_alt.savefig(f"{out_folder}\\{out_time}\\"*to_folder + f"{out_i}_dist_alt.png")
#                 if show_output:
#                     if graph:
#                         display(fig_Lq)
#                         print("\n\n")
#                     if graph_dist:
#                         display(fig_dist)
#                         print("\n\n\n")
#                         display(fig_dist_alt)
#                     pprint(avg_df)
#                     print("\n\n")
#                 out_i += 1


#                 plt.close("all")
                        
print("Run time:", int(time_lib.time()) - out_time, "seconds")

Good Case: Queue 0 is least expensive.
2999.5570435107015
Run time: 0 seconds
